Before we can test anything we first need to have some simple circuits to tests with. For this we create some simple circuits and use qiskit to write the equivalent qasm description for these.

In [ ]:
using PyCall

function create_qft_circuit(qubits::Integer)
    qiskit = pyimport("qiskit")
    qfts = pyimport("qiskit.aqua.components.qfts")

    circ = qiskit.QuantumCircuit(qubits)
    circ = qfts.Standard(qubits).construct_circuit(mode="circuit", 
                                                   qubits=circ.qregs[1],
                                                   circuit=circ)
    circ.qasm()
end

print(create_qft_circuit(3))

Write the QFT circuit on 3 qubits to a qasm file on disk

In [ ]:
for qubits in [2, 3, 5, 10] 
    circ_qasm = create_qft_circuit(qubits)
    open("../examples/qft_$(qubits).qasm", "w") do io
       write(io, circ_qasm)
    end
end

In [ ]:
;cat ../examples/qft_3.qasm

Create a GHZ circuit aswell.

In [ ]:
qiskit = pyimport("qiskit")
circ = qiskit.QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.draw()

In [ ]:
open("../examples/ghz_3.qasm", "w") do io
    write(io, circ.qasm())
end

In [ ]:
;cat ../examples/ghz_3.qasm

# Layer 3: circuit to tensor network

This layer has a few tasks
1. Load circuit from qasm
2. Transpile so that only neighbouring qubits interact
3. Construct tensor network
4. Save as usable format

## Task 1: load circuit from qasm

In [ ]:
using PyCall

qiskit = pyimport("qiskit")
circ = qiskit.QuantumCircuit.from_qasm_file("../examples/qft_3.qasm")
# circ = qiskit.QuantumCircuit.from_qasm_file("../examples/ghz_3.qasm")
# circ = qiskit.QuantumCircuit.from_qasm_file("../examples/qft_10.qasm")

In [ ]:
circ.draw()

## Task 2: Transpile

For the MPS network we require that there can only be two qubit gates acting between neighbouring qubits. Given a generic circuit which does not satisfy this we can enforce this by adding swap gates to move the interacting qubits together. The qiskit transpiler can do this for us.

In [ ]:
transpiler = pyimport("qiskit.transpiler")
passes = pyimport("qiskit.transpiler.passes")

coupling = [[i-1, i] for i = 1:circ.n_qubits]

coupling_map = transpiler.CouplingMap(PyCall.array2py([PyCall.array2py(x) for x in coupling]))

pass = passes.BasicSwap(coupling_map=coupling_map)
# pass = passes.LookaheadSwap(coupling_map=coupling_map)
# pass = passes.StochasticSwap(coupling_map=coupling_map)
pass_manager = transpiler.PassManager(pass)
transpiled_circ = pass_manager.run(circ)

transpiled_circ.draw()

In [ ]:
length(transpiled_circ.data)

## Task 3: Construct tensor network

We build the tensornetwork as a graph where each of the gates are nodes and addition nodes are added for start end end of qubits. We create a graph from the circuit. This proceeds by

1. Add node for each input qubit. This node will have a single index and the data will be initialised to [1,0] corresponding to state $|0\rangle$
2. Iterate over gates in the circuit and create nodes for each:
   a. Create node
   b. Reshape gate operation to appropriate shape and add
   c. Add indices to match shape order of data
   d. Add edges to existing nodes with edge dimensions
3. Add output nodes with corresponding 

In [ ]:
using LightGraphs, MetaGraphs
using JSON
using DataStructures
using Test

In [ ]:
struct TensorNetworkCircuit
    qubits::Integer
    graph::MetaDiGraph
    inputs::Array{Integer, 1}
    outputs::Array{Integer, 1}
end

"""
    function TensorNetworkCircuit(qubits::Integer)

Outer constructor to create an instance of TensorNetworkCircuit
"""
function TensorNetworkCircuit(qubits::Integer)
    graph = MetaDiGraph(qubits)
    # add input nodes
    inputs = collect(1:qubits)
    for i = 1:qubits
        set_props!(graph, i, Dict(:qubits => [i],
                                  :indices => [1],
                                  :dims => [2],
                                  :data => [1., 0.],
                                  :type => "input",
                                  :data_order => "col"))
    end
    
    # add output nodes
    outputs = collect((qubits + 1):(qubits * 2))
    for i = 1:qubits
        add_vertex!(graph)
        set_props!(graph, i + qubits, Dict(:qubits => [i + qubits],
                                  :indices => [1],
                                  :dims => [2],
                                  :data => [1., 0.],
                                  :type => "output",
                                  :data_order => "col"))
        # add edge from input to output
        add_edge!(graph, i, i + qubits)
        set_prop!(graph, i, i + qubits, :indices, [1, 1])
    end
    TensorNetworkCircuit(qubits, graph, inputs, outputs)
end

"""
    resize_gate(data, qubits)

Given the matrix corresponding to a gate, we reshape to have indices corresponding to qubits
"""
function resize_gate(data, qubits)
    if qubits == 1
        return data
    else
        @assert size(data) == (2^qubits, 2^qubits)
        return reshape(data, Tuple((2 for i = 1:4)))
    end
end

"""
    function add_gate!(tng::TensorNetworkCircuit, gate::Array{ComplexF64, 2}, qubits::Array{Integer, 1})

Add gate with matrix op given to circuit
"""
function add_gate!(tng::TensorNetworkCircuit, gate::Array{ComplexF64, 2}, qubits::Array{T, 1}) where T <: Integer
    # add a vertex for this gate
    add_vertex!(tng.graph)
    vertex_id = length(vertices(tng.graph))
    
    # prepare property values
    props = Dict{Symbol, Any}()
    props[:qubits] = qubits
    props[:indices] = collect(1:length(qubits)*2)
    props[:dims] = collect([2 for i = 1:length(qubits)*2])
    props[:data] = reshape(gate, prod(size(gate)))
    props[:type] = "gate"
    props[:data_order] = "col"
    
    set_props!(tng.graph, vertex_id, props)

    for (i, qubit) in enumerate(qubits)
        qubit_index = qubit + 1
        output_node = tng.outputs[qubit_index]
        from_node = inneighbors(tng.graph, output_node)[1]
        edge_indices = get_prop(tng.graph, from_node, output_node, :indices)
        rem_edge!(tng.graph, from_node, output_node)
        
        add_edge!(tng.graph, from_node, vertex_id)
        set_prop!(tng.graph, from_node, vertex_id, :indices, [edge_indices[1], i])
        
        add_edge!(tng.graph, vertex_id, output_node)
        set_prop!(tng.graph, vertex_id, output_node, :indices, [length(qubits) + i, edge_indices[2]])        
    end
end

Using this data structure and functions it makes it easy to create our tensor network graph

In [ ]:
tng = TensorNetworkCircuit(circ.n_qubits)
for gate in circ.data
    add_gate!(tng, gate[1].to_matrix(), [x.index for x in gate[2]])
end

## Task 4: Output in useable format 

We now write functions to serialise this data structure to json format.

In [ ]:
using JSON
using DataStructures

"""
    function to_json(tng::TensorNetworkCircuit, indent::Integer=0)

Convert the graph to a JSON string
"""
function to_json(tng::TensorNetworkCircuit, indent::Integer=0)
    top_level = OrderedDict()
    top_level["inputs"] = tng.inputs
    top_level["outputs"] = tng.outputs

    top_level["nodes"] = OrderedDict{Int64, Any}()
    nodes_dict = top_level["nodes"]
    for node in vertices(tng.graph)
        nodes_dict[node] = deepcopy(props(tng.graph, node))
        data = nodes_dict[node][:data]
        nodes_dict[node][:data_re] = reshape(real.(data), prod(size(data)))
        nodes_dict[node][:data_im] = reshape(imag.(data), prod(size(data)))
        delete!(nodes_dict[node], :data)
    end
    
    top_level["edges"] = OrderedDict{Int64, Any}()
    edges_dict = top_level["edges"]
    for (i, edge) in enumerate(edges(tng.graph))
        edges_dict[i] = props(tng.graph, edge)
        edges_dict[i][:src] = edge.src
        edges_dict[i][:dst] = edge.dst
    end
    
    JSON.json(top_level, indent)
end

And to deserialise.

In [ ]:
"""
    function from_json(json_str::String)

Convert a json formatted string to tensor network circuit graph
"""
function from_json(json_str::String)
    top_level = JSON.parse(json_str)
    nodes = top_level["nodes"]    
    
    graph = MetaDiGraph(length(nodes))
    for i = 1:length(nodes)
        node_dict = nodes["$(i)"]
        set_prop!(graph, i, :indices, convert(Array{Int64, 1}, node_dict["indices"]))
        set_prop!(graph, i, :dims, convert(Array{Int64, 1}, node_dict["dims"]))
        set_prop!(graph, i, :type, node_dict["type"])
        set_prop!(graph, i, :data_order, node_dict["data_order"])
        set_prop!(graph, i, :data, node_dict["data_re"] .+ node_dict["data_im"] .* 1im)
        set_prop!(graph, i, :qubits, convert(Array{Int64, 1}, node_dict["qubits"]))
    end
    
    edges = top_level["edges"]
    for i = 1:length(edges)
        edge = edges["$(i)"]
        src, dst = edge["src"], edge["dst"]
        add_edge!(graph, src, dst)
        set_prop!(graph, src, dst, :indices, convert(Array{Int64, 1}, edge["indices"]))
        set_prop!(graph, src, dst, :src, src)
        set_prop!(graph, src, dst, :dst, dst)        
    end

    TensorNetworkCircuit(length(top_level["inputs"]), graph, top_level["inputs"], top_level["outputs"])
end

We test this by serialising, deserialising and serialising again and making sure both serialised versions match.

In [ ]:
using Test

@test to_json(from_json(to_json(tng))) == to_json(tng)

Small example.

In [ ]:
qiskit = pyimport("qiskit")
circ = qiskit.QuantumCircuit(2)
circ.h(0)
circ.cz(0, 1)
convert circ.qasm()

In [ ]:
tng = TensorNetworkCircuit(circ.n_qubits)
for gate in circ.data
    add_gate!(tng, gate[1].to_matrix(), [x.index for x in gate[2]])
end
print(to_json(tng, 2))

It would also be nice to be able to visualise graphically. We can do this with TikzGraphs but available layouts are a little limited.

In [ ]:
using TikzGraphs
using TikzPictures

In [ ]:
# other layouts available are Layered, Spring and SimpleNecklace
tikz_graph = TikzGraphs.plot(Graph(tng.graph.graph), Layouts.SpringElectrical())
TikzPictures.save(PDF("graph"), tikz_graph)

# Layer 2: Tensor Network to contractions

In this layer, a tensor network description in input and a sequence of contractions is planned which should output the desired amplitudes. The prototype implementation will simply contract across from input nodes until it gets to the end, not performing any compression or slicing.

Tasks to be performed by prototype implementation of this layer
1. Load tensor network graph description
2. Iterate over network from input nodes computing the bond dimensions along the way and recording indices
3. Output the planned contractions

## Task 1: Load tensor network

## 2. Iterate over network from input nodes computing the bond dimensions along the way and recording indices

## 3. Output the planned contractions